In [2]:
from pyspark.sql import SparkSession
# load dataset
import pandas as pd
pd_attri = pd.read_csv('/usr/dataset/ml-20m/movies.csv')

In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkConf
sc = SparkContext("spark://39.99.129.214:7077", "FirstApp")
spark = SparkSession.builder.master("spark://39.99.129.214:7077").config(conf=SparkConf()).getOrCreate()


In [3]:
import numpy as np
values_attri =[[i,a] for i,a in zip(pd_attri['movieId'].values,pd_attri['genres'].values)]

In [4]:
rdd = sc.parallelize(values_attri)

In [5]:
rdd.collect()

[[1, 'Adventure|Animation|Children|Comedy|Fantasy'],
 [2, 'Adventure|Children|Fantasy'],
 [3, 'Comedy|Romance'],
 [4, 'Comedy|Drama|Romance'],
 [5, 'Comedy'],
 [6, 'Action|Crime|Thriller'],
 [7, 'Comedy|Romance'],
 [8, 'Adventure|Children'],
 [9, 'Action'],
 [10, 'Action|Adventure|Thriller'],
 [11, 'Comedy|Drama|Romance'],
 [12, 'Comedy|Horror'],
 [13, 'Adventure|Animation|Children'],
 [14, 'Drama'],
 [15, 'Action|Adventure|Romance'],
 [16, 'Crime|Drama'],
 [17, 'Drama|Romance'],
 [18, 'Comedy'],
 [19, 'Comedy'],
 [20, 'Action|Comedy|Crime|Drama|Thriller'],
 [21, 'Comedy|Crime|Thriller'],
 [22, 'Crime|Drama|Horror|Mystery|Thriller'],
 [23, 'Action|Crime|Thriller'],
 [24, 'Drama|Sci-Fi'],
 [25, 'Drama|Romance'],
 [26, 'Drama'],
 [27, 'Children|Drama'],
 [28, 'Drama|Romance'],
 [29, 'Adventure|Drama|Fantasy|Mystery|Sci-Fi'],
 [30, 'Crime|Drama'],
 [31, 'Drama'],
 [32, 'Mystery|Sci-Fi|Thriller'],
 [33, 'Adventure|Romance|IMAX'],
 [34, 'Children|Drama'],
 [35, 'Drama|Romance'],
 [36, 'Crim

In [6]:
#计算TF-IDF：
documents = rdd.map(lambda l: l[1].split("|"))
from pyspark.mllib.feature import HashingTF, IDF
hashingTF = HashingTF(numFeatures=1000)
tf = hashingTF.transform(documents)
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

In [7]:
from pyspark.mllib.feature import Normalizer
labels = rdd.map(lambda l: l[0])
features = tfidf

normalizer = Normalizer()
data = labels.zip(normalizer.transform(features))

In [8]:
data_collect = data.collect() 
print(data_collect)

[(1, SparseVector(1000, {371: 0.5387, 669: 0.194, 808: 0.4864, 829: 0.4042, 986: 0.5217})), (2, SparseVector(1000, {808: 0.5933, 829: 0.4931, 986: 0.6363})), (3, SparseVector(1000, {669: 0.5571, 916: 0.8305})), (4, SparseVector(1000, {669: 0.5278, 916: 0.7869, 968: 0.3196})), (5, SparseVector(1000, {669: 1.0})), (6, SparseVector(1000, {316: 0.527, 375: 0.6258, 546: 0.5751})), (7, SparseVector(1000, {669: 0.5571, 916: 0.8305})), (8, SparseVector(1000, {829: 0.6125, 986: 0.7905})), (9, SparseVector(1000, {546: 1.0})), (10, SparseVector(1000, {316: 0.5057, 546: 0.5519, 829: 0.6631})), (11, SparseVector(1000, {669: 0.5278, 916: 0.7869, 968: 0.3196})), (12, SparseVector(1000, {374: 0.8932, 669: 0.4496})), (13, SparseVector(1000, {371: 0.6323, 829: 0.4745, 986: 0.6124})), (14, SparseVector(1000, {968: 1.0})), (15, SparseVector(1000, {546: 0.5605, 829: 0.6735, 916: 0.4819})), (16, SparseVector(1000, {375: 0.9522, 968: 0.3056})), (17, SparseVector(1000, {916: 0.9265, 968: 0.3763})), (18, Spars

In [9]:
len(data_collect)

27278

In [10]:
rows,columns,values = [],[],[]
for line in data_collect:
    row = line[0]
    for i,j in zip(line[1].indices,line[1].values):
        rows.append(row)
        columns.append(i)
        values.append(j)
rows = np.array(rows)     
columns = np.array(columns)     
values = np.array(values,dtype=np.float16)     


In [11]:
n_user = rows.max()
n_item = columns.max()


In [1]:
from scipy import sparse
coo = sparse.coo_matrix((values,(rows,columns)),shape=(n_user+1,n_item+1),dtype=np.float16)
sparse.save_npz('./tfidf.npz',coo)

NameError: name 'values' is not defined

In [1]:
from scipy import sparse
import copy
coo = sparse.load_npz('./tfidf.npz')
lil = coo.tolil()
n_user,n_item = lil.shape

In [2]:
from tqdm import tqdm
import numpy as np
import copy
batch = 100
lil_right = lil.transpose()
topk = []
for i in tqdm(range(int(n_user/batch))):
    lil_left = lil[i*batch:(i+1)*batch]
    sim_lil = lil_left.dot(lil_right)  
    v = -sim_lil.toarray()
    top5 = copy.deepcopy(np.argsort(v)[:,:5])
    topk.append(top5)

100%|██████████| 1312/1312 [06:28<00:00,  3.38it/s]


In [5]:
topk_tl = np.concatenate(topk,axis=0) 
import pickle
pickle_topk = {i:line for i,line in enumerate(topk_tl)}
fw = open('item_similarity.pkl','wb')  
pickle.dump(topk_tl, fw, -1)  



In [12]:
import pymongo
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext

In [ ]:
client = pymongo.MongoClient("39.98.136.173", 9099)
client.movie.authenticate('user','cloud',mechanism='SCRAM-SHA-1')
database = client['movie']
ratings = database['ratings_sort']
cursor = ratings.find()
df = pd.DataFrame(list(cursor))
df = df[['userId', 'movieId', 'rating', 'timestamp']]

In [10]:
spark = SparkSession.builder.appName('test').getOrCreate()
sqlContest = SQLContext(spark)
spark_df = sqlContest.createDataFrame(df)
spark_df.show(10)

   userId  movieId  rating   timestamp
0       1      924     3.5  1094785598
1       1      919     3.5  1094785621
2       1     2683     3.5  1094785650
3       1     1584     3.5  1094785656
4       1     1079     4.0  1094785665
5       1      653     3.0  1094785691
6       1     2959     4.0  1094785698
7       1      337     3.5  1094785709
8       1     1304     3.0  1094785720
9       1     3996     4.0  1094785727
+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    924|   3.5|1094785598|
|     1|    919|   3.5|1094785621|
|     1|   2683|   3.5|1094785650|
|     1|   1584|   3.5|1094785656|
|     1|   1079|   4.0|1094785665|
|     1|    653|   3.0|1094785691|
|     1|   2959|   4.0|1094785698|
|     1|    337|   3.5|1094785709|
|     1|   1304|   3.0|1094785720|
|     1|   3996|   4.0|1094785727|
+------+-------+------+----------+

